In [ ]:
%load_ext watermark


In [ ]:
import os

from IPython.display import display
import matplotlib as mpl
import polars as pl
import seaborn as sns
from teeplot import teeplot as tp


In [ ]:
%watermark -diwmuv -iv


In [ ]:
tp.save[".pgf"] = True
teeplot_subdir = os.environ.get(
    "NOTEBOOK_NAME", "2025-02-19-cpp-bench-speed-pico"
)
teeplot_subdir


## Prep Data


In [ ]:
algo_names = {
    "dstream.circular_algo": "control: ringbuf",
    "control_throwaway_algo": "control: discard",
    "dstream.steady_algo": "dstream steady",
    "dstream_tilted_algo": "dstream tilted",
    "dstream.compressing_algo": "gunther steady",
    "doubling_steady_algo": "doubling steady",
    "doubling_tilted_algo": "doubling tilted",
    "zhao_tilted_algo": "zhao tilted",
    "zhao_steady_algo": "zhao steady",
}


df = pl.concat(
    [
        pl.read_csv("https://osf.io/mtguz/download"),
    ],
)
df = (
    df.cast(
        {
            "memory_bytes": pl.Int32,
            "num_items": pl.Int32,
            "num_sites": pl.Int32,
            "duration_s": pl.Float64,
            "replicate": pl.Int32,
        },
    )
    .with_columns(
        duration_per_item_ns=(
            pl.col("duration_s") * 1_000_000_000 / pl.col("num_items")
        )
    )
    .filter(
        pl.col("algo_name").is_in(
            algo_names.keys(),
        ),
    )
    .with_columns(
        algorithm=pl.col("algo_name").map_elements(
            algo_names.__getitem__,
            return_dtype=str,
        )
    )
    .with_columns(pl.col("data_type").alias("data type"))
)

display(df.describe()), display(df.head()), display(df.tail());


## Plot


In [ ]:
for rc in [{}, {"font.family": "serif"}]:
    with mpl.rc_context(rc=rc):
        with tp.teed(
            sns.relplot,
            data=df.with_columns(
                pl.col("num_sites").alias("buffer size $S$"),
                pl.col("duration_per_item_ns").alias("per-item walltime (ns)"),
                pl.col("num_items").alias("num items ingested"),
            ),
            x="buffer size $S$",
            y="per-item walltime (ns)",
            col="num items ingested",
            row="data type",
            hue="algorithm",
            style="algorithm",
            palette="muted",
            kind="line",
            aspect=1.5,
            markers=["^", "X"],
            ms=8,
            height=2,
            teeplot_outattrs=rc,
            teeplot_subdir=teeplot_subdir,
            facet_kws=dict(margin_titles=True),
        ) as g:
            g.set(xscale="log", yscale="log")
            sns.move_legend(
                g,
                "lower center",
                bbox_to_anchor=(0.35, 1),
                ncol=2,
                title=None,
                frameon=False,
            )
            g.tight_layout()


In [ ]:
for rc in [{}, {"font.family": "serif"}]:
    with mpl.rc_context(rc=rc):
        for num_sites in [64, 256, 1024, 4096]:
            with tp.teed(
                sns.relplot,
                data=df.filter(
                    pl.col("algorithm").is_in(
                        [
                            "control: ringbuf",
                            "control: discard",
                            "dstream tilted",
                            "gunther steady",
                            "naive tilted",
                            "zhao tilted",
                        ],
                    ),
                    pl.col("num_sites") == num_sites,
                ).with_columns(
                    pl.col("num_sites").alias("buffer size $S$"),
                    pl.col("duration_per_item_ns").alias("per-item walltime (ns)"),
                    pl.col("num_items").alias("num items ingested"),
                ),
                x="num items ingested",
                y="per-item walltime (ns)",
                col="data type",
                col_wrap=2,
                col_order=[
                    "bit",
                    "byte",
                    "word",
                    "double word",
                ],
                hue="algorithm",
                hue_order=[
                    "control: ringbuf",
                    "control: discard",
                    "dstream tilted",
                    "gunther steady",
                    "naive tilted",
                    "zhao tilted",
                ],
                style="algorithm",
                palette="muted",
                kind="line",
                aspect=1.5,
                markers=["^", "X", "P", "d", "*", "v"],
                ms=8,
                height=2,
                teeplot_subdir=teeplot_subdir,
                facet_kws=dict(margin_titles=False),
                teeplot_outattrs={
                    "num_sites": num_sites,
                    **rc,
                },
            ) as g:
                g.set(xscale="log", yscale="log")
                sns.move_legend(
                    g,
                    "upper center",
                    bbox_to_anchor=(0.42, 1.12),
                    ncol=3,
                    title=None,
                    frameon=False,
                )
                g.tight_layout()


In [ ]:
for rc in [{}, {"font.family": "serif"}]:
    with mpl.rc_context(rc=rc):
        with tp.teed(
            sns.relplot,
            data=df.filter(
                pl.col("algorithm").is_in(
                    [
                        "control: ringbuf",
                        "control: discard",
                        "dstream tilted",
                        "gunther steady",
                        "naive tilted",
                        "zhao tilted",
                    ],
                ),
                pl.col("num_items") == 100_000,
            ).with_columns(
                pl.col("num_sites").alias("buffer size $S$"),
                pl.col("duration_per_item_ns").alias("per-item\nwalltime (ns)"),
                pl.col("num_items").alias("num items ingested"),
            ),
            x="buffer size $S$",
            y="per-item\nwalltime (ns)",
            col="data type",
            col_wrap=2,
            col_order=[
                "bit",
                "byte",
                "word",
                "double word",
            ],
            hue="algorithm",
            hue_order=[
                "control: ringbuf",
                "control: discard",
                "dstream tilted",
                "gunther steady",
                "naive tilted",
                "zhao tilted",
            ],
            style="algorithm",
            palette="muted",
            kind="line",
            aspect=1.2,
            markers=["^", "X"],
            ms=8,
            height=1.8,
            teeplot_outattrs=rc,
            teeplot_subdir=teeplot_subdir,
            facet_kws=dict(margin_titles=False),
        ) as g:
            g.set(xscale="log", yscale="log")
            sns.move_legend(
                g,
                "upper center",
                bbox_to_anchor=(0.42, 1.22),
                ncol=2,
                title=None,
                frameon=False,
            )
            for ax in g.axes.flat:
                ax.set_xticks([64, 256, 1024, 4096])
                ax.set_xticklabels(["64", "256", "1024", "4096"])
            g.tight_layout()
